<a href="https://colab.research.google.com/github/MuhammadZunair19/AutoScroller/blob/main/Demo_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install selenium beautifulsoup4 pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 492.9/492.9 kB 15.7 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin

# Ask for the URL
base_url = input("Enter the URL of the page: ").strip()

try:
    # Make a GET request to fetch the raw HTML content
    response = requests.get(base_url)
    response.raise_for_status()  # Raise an error for bad responses

    # Parse the HTML content
    soup = BeautifulSoup(response.text, 'html.parser')

    # Find the table by id
    table = soup.find('table', {'id': 'agent-table'})

    if not table:
        print("No table with id 'agent-table' found!")
        exit()

    # Find all 'a' tags with class 'agent-detail' inside the table
    links = table.find_all('a', class_='agent-detail')

    if not links:
        print("No links with class 'agent-detail' found in the table.")
        exit()

    print("\nExtracted Full Detail Links:\n")
    for link in links:
        href = link.get('href')
        full_url = urljoin(base_url, href)  # Convert relative URLs to absolute
        print(full_url)

except requests.exceptions.RequestException as e:
    print(f"Error fetching the URL: {e}")


Enter the URL of the page: https://theppra.org.za/index.php?type=Agents&search_agent=%25&submit_agent_search=GO&data_url=%2Fagent_agency_search&startpage=2&startrow=0&startpage=1

Extracted Full Detail Links:

https://theppra.org.za/listing_detail.php?agents_id=23243&t=b1df7aa0bf58d581c57d816c85331347
https://theppra.org.za/listing_detail.php?agents_id=47492&t=13d8d6e805c96b2951fa97623591918b
https://theppra.org.za/listing_detail.php?agents_id=48718&t=6a4e587520981d9cb00fd1233fe2665a
https://theppra.org.za/listing_detail.php?agents_id=49180&t=5640f6f1e5900d2399c0570da3b9b1e8
https://theppra.org.za/listing_detail.php?agents_id=59098&t=ce207f5643d878dec9eeb4eefa361fd6
https://theppra.org.za/listing_detail.php?agents_id=59514&t=1b2cfa9caae2288540dbdf743e948ccf
https://theppra.org.za/listing_detail.php?agents_id=59519&t=d49237e18e2c362e765601fc04301639
https://theppra.org.za/listing_detail.php?agents_id=150208&t=f59152fc7a99aeb5b11df41ed0295578
https://theppra.org.za/listing_detail.php?age

In [ ]:
import requests
from bs4 import BeautifulSoup
import time
import csv

# Base URL of the site
BASE_URL = "https://www.eaab.org.za"

def get_agent_links(main_url):
    """
    Get all 'Full Detail' agent links from the main search page.
    """
    response = requests.get(main_url)
    soup = BeautifulSoup(response.content, "html.parser")

    links = []
    for a_tag in soup.find_all("a", class_="agent-detail"):
        href = a_tag.get("href")
        if href:
            full_url = BASE_URL + href
            links.append(full_url)

    print(f"Found {len(links)} agent links.\n")
    return links

def scrape_agent_details(detail_url):
    """
    Scrape agent details from a specific agent's detail page.
    """
    print(f"Scraping URL: {detail_url}")
    response = requests.get(detail_url)
    soup = BeautifulSoup(response.content, "html.parser")

    data_div = soup.find("div", class_="result-list default")
    agent_data = {}

    if data_div:
        rows = data_div.find_all("tr")

        # Grab the h3 heading (Full Name or Entity Name)
        heading = data_div.find("h3")
        if heading:
            agent_data["Full Name"] = heading.get_text(strip=True)

        # Loop through table rows to get the data
        for row in rows:
            caption_cell = row.find("td", class_="caption")
            if caption_cell:
                label = caption_cell.get_text(strip=True)
                value_cell = row.find_all("td")[1]
                value = value_cell.get_text(separator=" ", strip=True)

                # Match the labels to friendly keys
                if label == "Firstname:":
                    agent_data["Firstname"] = value
                elif label == "Lastname":
                    agent_data["Lastname"] = value
                elif label == "City":
                    agent_data["City"] = value
                elif label == "Reference Number":
                    agent_data["Reference Number"] = value
                elif label == "Agency Name":
                    agent_data["Agency Name"] = value
                elif label == "Trading As":
                    agent_data["Trading As"] = value
                elif label == "Designation":
                    agent_data["Designation"] = value
                elif label == "Physical Adress":
                    agent_data["Physical Address"] = value
                elif label == "Tel":
                    agent_data["Tel"] = value
                elif label == "Fax":
                    agent_data["Fax"] = value

        agent_data["Scraped URL"] = detail_url
    else:
        print("No agent data found in this page.")

    return agent_data

def save_to_csv(agent_data_list, filename="agent_data.csv"):
    """
    Save scraped agent data to a CSV file.
    """
    if not agent_data_list:
        print("No data to save.")
        return

    # Define all potential fields that might exist in your data
    fieldnames = [
        "Full Name", "Firstname", "Lastname", "City", "Reference Number",
        "Agency Name", "Trading As", "Designation", "Physical Address", "Tel", "Fax", "Scraped URL"
    ]

    with open(filename, mode="w", newline='', encoding="utf-8") as csv_file:
        writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
        writer.writeheader()

        for data in agent_data_list:
            # Make sure all fields are present; fill in missing fields with empty strings
            row = {field: data.get(field, "") for field in fieldnames}
            writer.writerow(row)

    print(f"\n✅ Data saved to {filename}")

def main():
    # Replace this with your actual search URL
    main_url = "https://theppra.org.za/index.php?type=Agents&search_agent=%25&submit_agent_search=GO&data_url=%2Fagent_agency_search&startpage=1&startrow=20&startpage=2"

    # Get all agent detail page links
    agent_links = get_agent_links(main_url)

    all_agents_data = []

    # Iterate over each link, scrape details, and add to the list
    for link in agent_links:
        agent_data = scrape_agent_details(link)
        if agent_data:  # Only add non-empty results
            all_agents_data.append(agent_data)
        time.sleep(1)  # Be polite; add a delay

    # Save the collected data to CSV
    save_to_csv(all_agents_data)

if __name__ == "__main__":
    main()


Found 20 agent links.

Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=125756&t=3ae98c962320b757fbb5f60cedacecef
Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=87142&t=27f51ec81eaf8650735c5f8adc7686fd
Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=53118&t=7b491727da003c89063df158f1302301
Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=14571&t=8fd3ecfa9f0d66afcea1eac57c7b7cae
Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=96735&t=16ddc0a3f0a24f137bc1855da00a2c3b
Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=103662&t=a0eca7dace294c2a8bfbdeb98db771ff
Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=141539&t=f3dca14a6862ed3e8a14c895ef10ed22
Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=36753&t=012f0c84923238e5b037de66b654c88c
Scraping URL: https://www.eaab.org.za/listing_detail.php?agents_id=1&t=bcc180d07bfda1bd5ba2d2dbc3e82ef5
Scrapi